In [281]:
import requests, json
import pandas as pd
import numpy as np
import time

In [207]:
# read excel with country names and codes
# return list with country codes relevant for UCDP
country_code_pathfile = '/Users/sabine.a.joseph/Documents/Country_Feature_documentation/Country_codes_NAMO.xlsx'
country_code_column_name = 'UCDP_country_codes'
sheet = 'Sheet1'

def country_codes_from_excel(country_codes, sheet_num, column_name):
    countries = pd.ExcelFile(country_codes)
    countries = countries.parse(sheet_num)
    return list(countries[column_name]) # UCDP uses Gleditsch and Ward country codes

CCs = country_codes_from_excel(country_code_pathfile, sheet, country_code_column_name)

In [409]:
### API call and data extraction # no longer in use
# prepare country filter
#CCs to comma seperated string

all_CCs = ''
for i in range(0, len(CCs)):
    if CCs[i].is_integer():
        all_CCs = all_CCs+','.join((str(int(CCs[i])), ''))
all_CCs = all_CCs[:-1]

years = list(np.linspace(1989, 2018, num=30))
months = list(np.linspace(1, 10, num=4)) 

#for i in range(0, len(years)-1):
for i in range(0, 1):
    start = time.time()
    start_str_y = str(int(years[i]))
    end_str_y = str(int(years[i]))
    
    for j in range(0, len(months)-1):
        if months[j] < 9:
            start_str_m = '0' + str(int(months[j]))
            end_str_m = '0' + str(int(months[j+1])) #elif
        else:    
            start_str_m = str(int(months[j]))
            end_str_m = str(int(months[j+1]))
        
        r = requests.get(web_service_url + page + '&Country=' + all_CCs 
                         + '&StartDate=' + start_str_y + '-' + start_str_m 
                         + '-01&EndDate=' + end_str_y + '-' + end_str_m + '-01') #date filter: YYYY-MM-DD
        
        end = time.time()
        print 'time elapsed'
        print(end - start)
        print web_service_url + page + '&Country=' + all_CCs + '&StartDate=' + start_str_y + '-' + start_str_m + '-01&EndDate=' + end_str_y + '-' + end_str_m + '-01'
        
        results = r.json()

        if i == 0:
            df = pd.DataFrame.from_dict(results['Result']) #create df on first loop iteration
        else:
            df = pd.concat([df, pd.DataFrame.from_dict(results['Result'])]) #concatenate df on each iteration
        
    df = pd.DataFrame.from_dict(results['Result'])


In [401]:
for i in range(0, len(df.columns)):
    #print df.columns[i]
    x = float(sum(df[df.columns[i]].value_counts()))/float(len(df.id))
    #print x